In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display

cwd = os.getcwd()

In [2]:
# read the new dataframe back into memory
df = pd.read_csv(os.path.join(cwd, "Citibike_Clean", "CitiBike-NYC-FINAL.csv.zip"),
                       compression="zip")

# change the datatype for several columns into categorical to save on RAM memory
df["Start Station ID"] = pd.Categorical(df["Start Station ID"])
df["End Station ID"] = pd.Categorical(df["End Station ID"])
df["Bike ID"] = pd.Categorical(df["Bike ID"])
df["User Type"] = pd.Categorical(df["User Type"])
df["Birth Year"] = pd.Categorical(df["Birth Year"])
df["Gender"] = pd.Categorical(df["Gender"])
df["Start Year"] = pd.Categorical(df["Start Year"])
df["Start Month"] = pd.Categorical(df["Start Month"])
df["Start Day"] = pd.Categorical(df["Start Day"])
df["Start Hour"] = pd.Categorical(df["Start Hour"])

df.head()

,Start Station ID,End Station ID,Bike ID,User Type,Birth Year,Gender,Start Year,Start Month,Start Day,Start Hour,Duration_Seconds,Station_Distance,Seconds_per_Metre
0,3226,3165,25542,Subscriber,1965.0,2,2017,1,1,0,680,875,0.78
1,3226,3165,24209,Customer,NaN,0,2017,1,1,14,1756,875,2.01
2,3226,3165,15708,Customer,NaN,0,2017,1,1,14,1750,875,2.00
3,3226,3165,18403,Customer,NaN,0,2017,1,1,14,1735,875,1.98
4,3226,3165,16505,Customer,NaN,0,2017,1,1,14,1691,875,1.93


# First Aggregation

- aggregate on Year, Month, Day and Hour
    - sum and count on Duration and Distance

In [3]:
# aggregate on year, month, day and hour for duration in seconds
df_agg = (df.loc[:, ['Start Year', 'Start Month', 'Start Day',
           'Start Hour', 'Duration_Seconds', 'Station_Distance']]
   .groupby(['Start Year', 'Start Month', 
             'Start Day', 'Start Hour']).agg(["sum", "count"]))
df_agg.columns = df_agg.columns.droplevel()
df_agg.columns = ["Duration_Seconds_SUM", "Duration_Seconds_COUNT",
                  "Station_Distance_SUM", "Station_Distance_COUNT"]
df_agg.reset_index(inplace=True)

# fill empty values with 0
df_agg.fillna(value={
    "Duration_Seconds_SUM":0,
    "Duration_Seconds_COUNT":0,
    "Station_Distance_SUM":1,
    "Station_Distance_COUNT":1
                    }, inplace=True)

# set the datatype for sum and count
df_agg["Duration_Seconds_SUM"] = df_agg["Duration_Seconds_SUM"].astype("int64")
df_agg["Duration_Seconds_COUNT"] = df_agg["Duration_Seconds_COUNT"].astype("int64")
df_agg["Station_Distance_SUM"] = df_agg["Station_Distance_SUM"].astype("int64")
df_agg["Station_Distance_COUNT"] = df_agg["Station_Distance_COUNT"].astype("int64")

# write this to csv file
df_agg.to_csv(os.path.join(cwd, "Citibike_Aggregate", "GroupBy_Year_Month_Day_Hour_Seconds_Distance.csv"),
                 index=False)

df_agg.head()

,Start Year,Start Month,Start Day,Start Hour,Duration_Seconds_SUM,Duration_Seconds_COUNT,Station_Distance_SUM,Station_Distance_COUNT
0,2017,1,1,0,654326,706,1320474,706
1,2017,1,1,1,519550,704,1084164,704
2,2017,1,1,2,543026,564,1000274,564
3,2017,1,1,3,290928,414,614700,414
4,2017,1,1,4,567570,192,273208,192


# Second Aggregation

- aggregate on year, month, day, hour , user type and gender
    - sum and count on duration and distance

In [4]:
del df_agg

# year, month, day, and hour, user type, gender groupby
df_agg = (df.loc[:, ['Start Year', 'Start Month', 'Start Day',
           'Start Hour', 'User Type', 'Gender', 'Duration_Seconds', 'Station_Distance']]
   .groupby(['Start Year', 'Start Month', 
             'Start Day', 'Start Hour',
             'User Type', 'Gender']).agg(["sum", "count"]))
df_agg.columns = df_agg.columns.droplevel()
df_agg.columns = ["Duration_Seconds_SUM", "Duration_Seconds_COUNT",
                  "Station_Distance_SUM", "Station_Distance_COUNT"]
df_agg.reset_index(inplace=True)

# fill empty values with 0
df_agg.fillna(value={
    "Duration_Seconds_SUM":0,
    "Duration_Seconds_COUNT":0,
    "Station_Distance_SUM":1,
    "Station_Distance_COUNT":1
                    }, inplace=True)

# set the datatype for sum and count
df_agg["Duration_Seconds_SUM"] = df_agg["Duration_Seconds_SUM"].astype("int64")
df_agg["Duration_Seconds_COUNT"] = df_agg["Duration_Seconds_COUNT"].astype("int64")
df_agg["Station_Distance_SUM"] = df_agg["Station_Distance_SUM"].astype("int64")
df_agg["Station_Distance_COUNT"] = df_agg["Station_Distance_COUNT"].astype("int64")

# write this to csv file
df_agg.to_csv(os.path.join(cwd, "Citibike_Aggregate", "GroupBy_Year_Month_Day_Hour_User_Gender_Seconds_Distance.csv"),
                 index=False)

df_agg.head()

,Start Year,Start Month,Start Day,Start Hour,User Type,Gender,Duration_Seconds_SUM,Duration_Seconds_COUNT,Station_Distance_SUM,Station_Distance_COUNT
0,2017,1,1,0,Customer,0,188340,114,221436,114
1,2017,1,1,0,Customer,1,1080,2,2056,2
2,2017,1,1,0,Customer,2,0,0,1,0
3,2017,1,1,0,Subscriber,0,1416,6,3246,6
4,2017,1,1,0,Subscriber,1,346888,450,836554,450


# Third Aggregation

- aggregate on Bike ID only
    - sum and count on duration and station

In [5]:
del df_agg

# bike id groupby
df_agg = (df.loc[:, ["Bike ID", 'Duration_Seconds', 'Station_Distance']]
   .groupby(['Bike ID']).agg(["sum", "count"]))
df_agg.columns = df_agg.columns.droplevel()
df_agg.columns = ["Duration_Seconds_SUM", "Duration_Seconds_COUNT",
                  "Station_Distance_SUM", "Station_Distance_COUNT"]
df_agg.reset_index(inplace=True)

# fill empty values with 0
df_agg.fillna(value={
    "Duration_Seconds_SUM":0,
    "Duration_Seconds_COUNT":0,
    "Station_Distance_SUM":1,
    "Station_Distance_COUNT":1
                    }, inplace=True)

# set the datatype for sum and count
df_agg["Duration_Seconds_SUM"] = df_agg["Duration_Seconds_SUM"].astype("int64")
df_agg["Duration_Seconds_COUNT"] = df_agg["Duration_Seconds_COUNT"].astype("int64")
df_agg["Station_Distance_SUM"] = df_agg["Station_Distance_SUM"].astype("int64")
df_agg["Station_Distance_COUNT"] = df_agg["Station_Distance_COUNT"].astype("int64")

# sort the aggregation so the largest values are on top
df_agg.sort_values(by=['Duration_Seconds_SUM'], ascending=False, inplace=True)
df_agg.reset_index(drop=True, inplace=True)

# write this to csv file
df_agg.to_csv(os.path.join(cwd, "Citibike_Aggregate", "GroupBy_bikeid_Seconds_Distance.csv"),
                 index=False)

# grab the first 50 bike ID then slice the dataframe down
df_top_50_bikes = df_agg["Bike ID"][:50]
df_top_50 = df.loc[(df["Bike ID"].cat.codes.isin(df_top_50_bikes.cat.codes)), :].copy()
df_top_50["Bike ID"] = df_top_50["Bike ID"].astype("int64")

df_agg.head()

,Bike ID,Duration_Seconds_SUM,Duration_Seconds_COUNT,Station_Distance_SUM,Station_Distance_COUNT
0,30503,22496822,3529,6208666,3529
1,30875,18806642,3342,5940214,3342
2,16777,18304323,2968,4847717,2968
3,18504,16576080,2782,4672241,2782
4,16533,16514771,3071,5372310,3071


In [6]:
# make sure the datatype is correct (no residual memory of other bikes)
df_top_50.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156266 entries, 197 to 55187233
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Start Station ID   156266 non-null  category
 1   End Station ID     156266 non-null  category
 2   Bike ID            156266 non-null  int64   
 3   User Type          156194 non-null  category
 4   Birth Year         150216 non-null  category
 5   Gender             156266 non-null  category
 6   Start Year         156266 non-null  category
 7   Start Month        156266 non-null  category
 8   Start Day          156266 non-null  category
 9   Start Hour         156266 non-null  category
 10  Duration_Seconds   156266 non-null  int64   
 11  Station_Distance   156266 non-null  int64   
 12  Seconds_per_Metre  156266 non-null  float64 
dtypes: category(9), float64(1), int64(3)
memory usage: 7.7 MB


In [7]:
# verify only top 50 bikes are present
len(df_top_50['Bike ID'].unique())

50

# Fourth Aggregation

- groupby on year, month, day, hour, bike ID for only the top 50 bikes
    - sum and count on duration and distance

In [8]:
del df_agg, df

# year, month, day, and hour, top 50 bike id groupby
df_agg = (df_top_50.loc[:, ["Start Year", "Start Month", "Start Day", "Start Hour",
                            "Bike ID", 'Duration_Seconds', 'Station_Distance']]
   .groupby(["Start Year", "Start Month", "Start Day", "Start Hour",
             'Bike ID']).agg(["sum", "count"]))
df_agg.columns = df_agg.columns.droplevel()
df_agg.columns = ["Duration_Seconds_SUM", "Duration_Seconds_COUNT",
                  "Station_Distance_SUM", "Station_Distance_COUNT"]
df_agg.reset_index(inplace=True)

# fill empty values with 0
df_agg.fillna(value={
    "Duration_Seconds_SUM":0,
    "Duration_Seconds_COUNT":0,
    "Station_Distance_SUM":1,
    "Station_Distance_COUNT":1
                    }, inplace=True)

# set the datatype for sum and count
df_agg["Duration_Seconds_SUM"] = df_agg["Duration_Seconds_SUM"].astype("int64")
df_agg["Duration_Seconds_COUNT"] = df_agg["Duration_Seconds_COUNT"].astype("int64")
df_agg["Station_Distance_SUM"] = df_agg["Station_Distance_SUM"].astype("int64")
df_agg["Station_Distance_COUNT"] = df_agg["Station_Distance_COUNT"].astype("int64")

# write this to csv file
df_agg.to_csv(os.path.join(cwd, "Citibike_Aggregate", "GroupBy_Year_Month_Day_Hour_top50_bikeid_Seconds_Distance.csv"),
                 index=False)

display(df_agg.head())

del df_agg

,Start Year,Start Month,Start Day,Start Hour,Bike ID,Duration_Seconds_SUM,Duration_Seconds_COUNT,Station_Distance_SUM,Station_Distance_COUNT
0,2017,1,1,0,15336,0,0,1,0
1,2017,1,1,0,15356,0,0,1,0
2,2017,1,1,0,15411,0,0,1,0
3,2017,1,1,0,15795,0,0,1,0
4,2017,1,1,0,16355,0,0,1,0
